# Warmup: Counter. 

Count how many times each element in a list occurs.

```
[1, 3, 2, 1, 5, 3, 5, 1, 4] ⇒

    1: 3 times
    2: 1 time
    3: 2 times
    4: 1 time
    5: 2 times
```


In [105]:
from collections import Counter
# declaring the list 
l = [1, 3, 2, 1, 5, 3, 5, 1, 4]
# driver program 
d = Counter(l) 
for i in set(l):
    if (d[i] > 1) :
        print('{} :  {} times'.format(i, d[i])) 
    else:
        print('{} :  {} time'.format(i, d[i])) 

1 :  3 times
2 :  1 time
3 :  2 times
4 :  1 time
5 :  2 times


# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [125]:
def safe_dict(d,k):
    try:
        result = d[k]
    except KeyError as ke:
        result = 0
    return result

In [128]:
print(safe_dict({1:2,3:4},1))
print(safe_dict({1:2,3:4},'cat'))

2
0


# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentionned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [47]:
'''
open the file
read contecnt of the file to string
then get number of accurences 
'''
def count_hamlet() :
    occurrances=0
    f = open("data\hamlet.txt","r")
    data = f.read().upper()
    occurrances = data.count("HAMLET")
    print(f"Number of occurrences of the word HAMLET is: {occurrances}")
    #f.close
    return occurrances

In [51]:
print(count_hamlet())

Number of occurrences of the word HAMLET is: 474
474


### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [52]:

import find_hamlet
count_hamlet()

Number of occurrences of the word HAMLET is: 474


474

### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [6]:
def count_unique_words() :
    file =  open("data\hamlet.txt","r")
    lines = file.read().splitlines()

    uniques = set()
    for line in lines:
        uniques |= set(line.split())

    return print(f"Unique words: {len(uniques)}")

In [7]:
print(count_unique_words())

Unique words: 7676
None


In [1]:
import re
regex = re.compile('\s+')

def uniquewords(datafile):
    data_file = open(datafile, 'r')
    readfile = data_file.read().replace('\n', '')
    count_unique = len(set(regex.split(readfile)))
    data_file.close()
    return count_unique

uniquewords('data/hamlet.txt')

9219

# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [48]:
import os
count=0
for file in os.listdir("..\data\csrgraph"):
    if file.endswith(".py"):
        count += 1
print(count)

8


### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [ ]:
#that import all of them or one of them

In [70]:
'''
loop packages
loop files for each package
search for the packed in the file

'''
def countimportfiles(packages):
    path= "..\data\csrgraph"
    fileslist = os.listdir(path)
    count= 0
    for p in packages:
        count = 0
        for f in fileslist:
            if f.endswith(".py"):
                file =  open(path + '/' + f,"r")
                openfile = file.read()
                if  openfile.find('import '+p) != -1 or openfile.find('from '+p) != -1:
                    count += 1
                
        print (f"{p} :   {str(count)}" )

print(countimportfiles(["pandas","numpy","numba","sklearn"]))    

pandas :   4
numpy :   6
numba :   6
sklearn :   4
None


# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

In [164]:
import numpy as np

def IDF(docs):
    #get the lsit of unqiue word
    u_words = np.unique(docs)
    N = len(docs)
    ln=len(u_words)
    #loop unique words
    word_document_frequency = []
    for uw in u_words:

        #loop documents
        uw_count=0
        for d in docs:
            if bool(np.intersect1d(uw,d)) == True:
                uw_count += 1
            
        word_document_frequency.append((uw,uw_count))
    
    for w in range(len(word_document_frequency)):
        result = np.log(N/(1 + (word_document_frequency[w][1])))
        print(f"{word_document_frequency[w][0]} : {result}")
  
IDF([['interview', 'questions'], ['interview', 'answers']])

answers : 0.0
interview : -0.40546510810816444
questions : 0.0


# Stretch Goal: IDF on Hamlet

Calculate the IDF dictionary on the Hamlet book.

What's the IDF of "Hamlet"?

What's the word with the highest IDF in the book?